In [ ]:
import os
import numpy as np 
from itertools import combinations
import tensorflow as tf
#import csv # to read string (color_*.jpg)
#from IPython.display import clear_output

np.random.seed(7)
np.set_printoptions(precision=4, suppress=True)  # 과학적 표기 대신 소숫점 자리 4자리까지 표현

rootDir = "D:/바탕화면/Dataset/TUG/trainSet"
expertFolder = "/" + "2021_01_24_saveResults_최윤정"  # --- 변경 할 부분
viewFolder = "/" + "0_sideView"

print(" [Home Dir.] Current path", os.getcwd())  

datasetDir = rootDir + expertFolder + viewFolder
os.chdir(datasetDir) ; print(" Current workaing path [dataset] ", os.getcwd())  # -- Dataset 있는 곳으로 경로 변경 

In [2]:
def trainData_split(datasetDir,train_ratio = 0.7, shuffle = False,  train_Kfold = False):      
   # print(next(os.walk(datasetDir))[1]) #['2020_11_03', '2020_11_20_v1', '2020_11_20_v2', '2020_11_23', '2020_12_09', '2020_12_30']
    expDates = next(os.walk(datasetDir))[1]  
    
    subject_number = 0
    subjects_list = []
    
    for dateFolder in expDates:
        dateDir = os.path.join(datasetDir, dateFolder)   # D:/바탕화면/Dataset/TUG/trainSet/2021_01_24_saveResults_최윤정/0_sideView\2020_11_03
        tmpSubjects = next(os.walk(dateDir))[1]
        subjects_list.append(tmpSubjects)  
        subject_number += len(tmpSubjects)
#     print(subjects_list)      # [['bys_tug', 'cbd_tug', 'cyj_tug'], ['kw', 'kyh', 'lhs', 'NHJ_60', 'nhs', 'pjh', 'pss', 'rjh', 'yjh']]
#     print(subject_number)     25

    # ------ 2D -> 1D [[sub1, sub2], [sub3, sub4]] -> [sub1, sub2, sub3, sub4]
    subjects = []
    for eachSub in subjects_list:
        subjects += eachSub
    print(" [before shuffle] 1D sub list: " , subjects)

    if shuffle:
        np.random.shuffle(subjects)
        print(" [After shuffle] 1D sub list: " , subjects)
        
    # ---- Split train/test subjects
    trainSub_number = np.round( (len(subjects)*train_ratio), 0).astype(int)
    testSub_number = subject_number - trainSub_number
    print("\t--> # of train sub: " + str(trainSub_number) +"명  ,  # of test subjects: " + str(testSub_number) + "명")
    
    
    
    train_Fold = list(combinations(subjects, trainSub_number))  # 25개 중 18개 뽑는 경우의 수 ==  25! / ((25 - 18)! * 18!) = 480700 개의 조합                     
    if train_Kfold:
        print(" [K-fold] combinations( ", subject_number, ",", trainSub_number, " )")
        return train_Fold
    else:
        return train_Fold[0]

train_subjects = trainData_split(datasetDir,train_ratio = 0.7, shuffle = False, train_Kfold = False)  

# # #     임의로 train, test set 만들기
# train_subjects =  ['NHJ_60',  'yjh', 'kyh', 'pss' ,'cyj_tug', 'cbd', 'cyj2', 'bys', 'jdh', 'jek', 'kch', 'kw', 'nhs', 'rjh'] 
# normal_subjects = ['lhs', 'pjh', 'cbd_tug', 'bys_tug','cyj','JDW_tug_fixed']
# patients_subjects = ['JDW_tug_fixed']
# test_subjects = patients_subjects # patients_subjects # normal_subjects

 [before shuffle] 1D sub list:  ['bys_tug', 'cbd_tug', 'cyj_tug', 'kw', 'kyh', 'lhs', 'NHJ_60', 'nhs', 'pjh', 'pss', 'rjh', 'yjh', 'bys', 'cbd', 'cyj', 'cyj2', 'jdh', 'jek', 'kch', 'ryu', 'JDW_tug', 'BYS_TUG', 'CYJ_TUG', 'PHE_TUG', 'YSJ_TUG']
	--> # of train sub: 18명  ,  # of test subjects: 7명


In [4]:
numFeature = 3
numActions = 5

def getPelvisData(csvFile):
    rawData = np.loadtxt(csvFile, delimiter=",")
    timestamp = rawData[:,0]
    pelvis_x = rawData[:,1]
    pelvis_y = rawData[:,2]
    pelvis_z = rawData[:,3]
    actionList = rawData[:,4:9]
    return timestamp, pelvis_x, pelvis_y, pelvis_z, actionList

def create_dataset(train_subjects):
    train_x = np.zeros((0, numFeature))
    train_y = np.zeros((0, numActions))
    test_x = np.zeros((0, numFeature))
    test_y = np.zeros((0, numActions))    
    
    trainSet = None
    testSet = None
    
    for dirpath, foldername, files in os.walk(datasetDir):
        for filename in files:
            if "lpf_" in filename:
                subname = dirpath.split("\\")[2] 
                csvFile = os.path.join(dirpath, filename)
                timestamp, pelvis_x, pelvis_y, pelvis_z, actionList = getPelvisData(csvFile) 
                
                pelvisData = np.array([pelvis_x, pelvis_y, pelvis_z]).T
                actionData = np.array(actionList)
                dataset = tf.data.Dataset.from_tensor_slices( (pelvisData.astype('float32'), actionData.astype('float32')) ) 
                
                
                if subname in train_subjects:
                    train_x = np.append(train_x, pelvisData, axis = 0 )
                    train_y = np.append(train_y, actionData, axis = 0 )

                    if trainSet == None:
                        trainSet = dataset
                    else:
                        trainSet = trainSet.concatenate(dataset)  
                else:
#                 if subname in test_subjects:
                    test_x = np.append(test_x, pelvisData, axis = 0 )
                    test_y = np.append(test_y, actionData, axis = 0 )    

                    if testSet == None:
                        testSet = dataset
                    else:
                        testSet = testSet.concatenate(dataset)

    return train_x, train_y, test_x, test_y
         
train_x, train_y, test_x, test_y = create_dataset(train_subjects)
print(" Train Shape X , Y: {0} , {1} \n Test Shape X , Y: {2} , {3}".format(train_x.shape, train_y.shape, test_x.shape, test_y.shape))

 Train Shape X , Y: (69605, 3) , (69605, 5) 
 Test Shape X , Y: (36119, 3) , (36119, 5)


In [5]:
def time_series_to_section(data_x, data_y, numActions, sliding_window_size, step_size_of_sliding_window, standardize=False, **options):
    data = data_x  # sensor data
    act_labels = data_y  # action labels
    mean = 0
    std = 1
    
    trainSet = None
    testSet = None
    if standardize:
            # As usual, normalize test dataset by training dataset's parameters
        if options:
            mean = options.get("mean")
            std = options.get("std")
                # \n test mean: {0}, std: {1}\n\n".format(mean,std))
            print("Test Data has been standardized:")
        else:
                # csv 한 줄의 mean ..이걸 모든 sensor data들에 대해 수행
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("Training Data has been standardized:\n the mean is = ", str(mean.mean()), " ; and the std is = ", str(std.mean()))
            # mean removal and variance scaling
        data -= mean
        data /= std
    else:
        print("----> Without Standardization.....")

        # We want the Rows of matrices show each Feature and the Columns show time points.
        # before transepose,,(145687, 12) -->  After (12, 145687)
    data = data.T

    size_features = data.shape[0]  # 4
    size_data = data.shape[1]

    number_of_secs = round( ((size_data - sliding_window_size)/step_size_of_sliding_window) )

        # Create a 3D matrix for Storing Snapshots
    secs_data = np.zeros((number_of_secs, size_features, sliding_window_size))
    act_secs_labels = np.zeros((number_of_secs, numActions))
    print("number of sections: ", number_of_secs, "size of data", size_data )
    k = 0
    for i in range(0, (size_data) - sliding_window_size, step_size_of_sliding_window):
        j = i // step_size_of_sliding_window # 0, 3, 6, 12
        if(j >= number_of_secs):
            break

        if(not (act_labels[i] == act_labels[i+sliding_window_size-1]).all()):
            continue

        secs_data[k] = data[0:size_features, i:i+sliding_window_size]
        act_secs_labels[k] = act_labels[i].astype(int)
        k = k+1
    secs_data = secs_data[0:k]    
    act_secs_labels = act_secs_labels[0:k]

    return secs_data, act_secs_labels, mean, std

In [6]:
sliding_window_size = 15 # 30.. 1sec.. 50 Equals to 1 sec for MotionSense Dataset (it is on 50Hz samplig rate)
step_size_of_sliding_window = 3 #10 
print("--> Sectioning Training and Test datasets: shape of each section will be: (", numFeature, "x", sliding_window_size, ")")

train_x_secs, train_y_secs, train_mean, train_std = time_series_to_section(train_x.copy(), train_y.copy(), numActions, sliding_window_size,step_size_of_sliding_window, standardize = True)

test_x_secs, test_y_secs, test_mean, test_std = time_series_to_section(test_x.copy(), test_y.copy(), numActions, sliding_window_size, step_size_of_sliding_window, standardize = True, mean = train_mean, std = train_std)

print("\n\n---> Training X Section: {0} , Training Label Section: {1} \n---> Test X Sections: {2} , Test Label Sections: {3}".format(train_x_secs.shape, train_y_secs.shape, test_x_secs.shape, test_y_secs.shape))

--> Sectioning Training and Test datasets: shape of each section will be: ( 3 x 15 )
Training Data has been standardized:
 the mean is =  0.4438291443250706  ; and the std is =  0.26557480458427785
number of sections:  23197 size of data 69605
Test Data has been standardized:
number of sections:  12035 size of data 36119


---> Training X Section: (18206, 3, 15) , Training Label Section: (18206, 5) 
---> Test X Sections: (10753, 3, 15) , Test Label Sections: (10753, 5)


In [7]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D , MaxPooling2D, Dropout

## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
train_x_block = np.expand_dims(train_x_secs, axis=3)
test_x_block = np.expand_dims(test_x_secs, axis=3)
print("---> Training Section: {0} , Test Sections: {1}".format(train_x_block.shape, test_x_block.shape))

---> Training Section: (18206, 3, 15, 1) , Test Sections: (10753, 3, 15, 1)


In [8]:
num_train, height, width, channel = train_x_block.shape  #   (13298, 4, 15, 1) =  (number_of_secs , size_features , sliding_window_size, 1)
metrics = ['acc']

## MTCNN
drop_prob_1 = 0.2 
drop_prob_2 = 0.4 


## Action model 
action_layer_dim = numActions # 4  # dws, ups, wlk, jog # , sit,  std 
action_activation_func = 'softmax'
action_loss_func = 'categorical_crossentropy'


def activityModel():
    input_layer = Input(shape = (height, width, 1) ) # (4, 15, 1) = (feature, sliding_window, 1)

    conv_0 = Conv2D(15, kernel_size=(1, 5), strides=(1, 1), padding='valid',  activation='relu') (input_layer) #5
    conv_1 = Conv2D(15, kernel_size=(1, 5), strides=(1, 1), padding='same', activation='relu') (conv_0)
    dense_1 = Dense(50, activation='relu') (conv_1)
    pool_1 = MaxPooling2D(pool_size=(1,2)) (dense_1)
    drop_1 = Dropout(drop_prob_1)(pool_1)


    conv_2 = Conv2D(11, kernel_size=(1, 3), strides=(1, 1), padding='valid',  activation='relu') (drop_1)# 5
    dense_2 = Dense(40, activation='relu') (conv_2)
    pool_2 = MaxPooling2D(pool_size=(1, 3)) (dense_2)  
    drop_2 = Dropout(drop_prob_1) (pool_2)

    conv_3 = Conv2D(10, kernel_size=(1, 3), strides=(1, 1), padding='same',  activation='relu') (drop_2)
    drop_3 = Dropout(drop_prob_1)(conv_3)

    flat = Flatten() (drop_3)
    hidden = Dense(400, activation = 'relu') (flat)
    drop_4 = Dropout(drop_prob_2)(hidden)

    action_out = Dense(action_layer_dim, activation = action_activation_func, name = 'ACTION') (drop_4)
    
    # --------- Model input/ Output Definition
    activity_model = Model(inputs=input_layer, outputs = action_out)
    activity_model.summary()

    activity_model.compile(loss=action_loss_func, optimizer=keras.optimizers.Adam(0.001), metrics=metrics)

    return activity_model

In [9]:
## Training Phase
batch_size = 64
num_of_epochs = 30 # 30
verbosity = 1

# Model Training 
model = activityModel()
history = model.fit(train_x_block, train_y_secs,                
              batch_size = batch_size,
              epochs = num_of_epochs,
              verbose = verbosity) 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 15, 1)]        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 3, 11, 15)         90        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 11, 15)         1140      
_________________________________________________________________
dense (Dense)                (None, 3, 11, 50)         800       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 5, 50)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 5, 50)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 11)         

In [10]:
#------------- Evaluation 
results_1 = model.evaluate(test_x_block, test_y_secs, verbose = verbosity)
print("--> Evaluation on Test Dataset:")
print("**** Accuracy for Activity Recognition task is: ", results_1)


# ----------------- 
y_predict = model.predict(test_x_block) # (6161, 5)  
y_pred_index = y_predict.argmax(axis=1) # print(y_pred_index) 
y_true_idx = np.argmax(test_y_secs, axis=1) # print(y_true_idx)

correct = np.nonzero(y_true_idx == y_pred_index) # print(correct, len(correct[0]))
incorrect = np.nonzero(y_true_idx != y_pred_index) # print(incorrect, len(incorrect[0]))

valid_num = len(correct[0]) + len(incorrect[0])
tot_num = len(y_true_idx)
# print("[must same] # of real data: ", tot_num, "==  # of pred data: ", valid_num)
percent_correct = len(correct[0]) /tot_num
percent_incorrect = len(incorrect[0])/tot_num
print(" correct: ", percent_correct," [%]", ",  incorrect: ", percent_incorrect," [%]")

337/337 [==============================] - 0s 650us/step - loss: 0.8603 - acc: 0.8600
--> Evaluation on Test Dataset:
**** Accuracy for Activity Recognition task is:  [0.8603415489196777, 0.8600390553474426]
 correct:  0.8600390588672928  [%] ,  incorrect:  0.13996094113270716  [%]


In [ ]:
tX = test_x.copy()
tYact = test_y

import matplotlib.pyplot as plt
print(y_pred_index)
print(y_predict)


# # print(y_predict[y_pred_index])
# raw_gyro = y_predict[np.argmax(y_pred_index)]
# plt.plot( y_pred_index, "b--", linewidth=0.5, markersize=1)    